In [1]:
import multiprocessing
import time

data = (
    ['a', '.2'], ['b', '.4'], ['c', '.6'], ['d', '.8'],
    ['e', '.1'], ['f', '.3'], ['g', '.5'], ['h', '.7']
)

def mp_worker((inputs, the_time)):
    print " Processs %s\tWaiting %s seconds" % (inputs, the_time)
    time.sleep(float(the_time))
    print " Process %s\tDONE" % inputs
    return inputs + inputs

def mp_handler():
    p = multiprocessing.Pool(2)
    return p.map(mp_worker, data)

In [2]:
mp_handler()

 Processs a	Waiting .2 seconds
 Processs b	Waiting .4 seconds
 Process a	DONE
 Processs c	Waiting .6 seconds
 Process b	DONE
 Processs d	Waiting .8 seconds
 Process c	DONE
 Processs e	Waiting .1 seconds
 Process e	DONE
 Processs f	Waiting .3 seconds
 Process d	DONE
 Processs g	Waiting .5 seconds
 Process f	DONE
 Processs h	Waiting .7 seconds
 Process g	DONE
 Process h	DONE


['aa', 'bb', 'cc', 'dd', 'ee', 'ff', 'gg', 'hh']

In [3]:
import numpy as np

def mp_worker(power):
    d = np.random.randn(100000000)**power
    print("Raising random array to the {0}th power".format(power))
    return d.mean()

data = [1, 2, 12, 15]

In [4]:
mp_handler()

Raising random array to the 2th power
Raising random array to the 1th power
Raising random array to the 12th power
Raising random array to the 15th power


[1.7296820979593759e-06,
 1.0000357408087308,
 10369.594516305644,
 -16222.247283663048]

In [5]:
import threading
import Queue

In [6]:
q = Queue.Queue()

In [7]:
q.put('foo')

In [8]:
q.put(5)

In [9]:
q.put('even more')

In [13]:
q.get(block=False)

Empty: 

In [14]:
def work():
    q.put(np.random.randn(1000))

In [15]:
t = threading.Thread(target=work)

In [16]:
t.start()

In [17]:
q.get().mean()

0.017436705760718853

In [23]:
t.